In [2]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

In [3]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        Product_Title = soup.find("h1", attrs={'class':'pdp-e-i-head'}).string.strip()
        
    except AttributeError:
        title_string = ""

    return Product_Title

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs = {'class':'payBlkBig'}).text

    except AttributeError:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("span", attrs = {'class':'avrg-rating'}).text
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':"total-rating showRatingTooltip"}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Product Description
def get_product_description(soup):
    try:
        Product_Description = soup.find_all("div", attrs={'class':'detailssubbox'})
        
    except AttributeError:
        Product_Description = ""	

    return Product_Description[1].text.strip()

# Function to extract Seller name
def get_seller_name(soup):
    try:
        Product_Review = soup.find("div", {'class':'sellerNameContainer h2 blackText'})
        Seller_Name=Product_Review.find("span").text
        
    except AttributeError:
        Seller_Name= ""

    return Seller_Name

In [11]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.snapdeal.com/search?clickSrc=top_searches&keyword=sunglasses&categoryId=0&vertical=p&noOfResults=20&SRPID=topsearch&sort=rlvncy"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    link1 = soup.find_all("a", {'class':'dp-widget-link noUdLine'})
    links=link1[0:40:2]

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "description":[],"sellername":[]}
    
     # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get(link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['description'].append(get_product_description(new_soup))
        d['sellername'].append(get_seller_name(new_soup))

    
    snapdeal_df = pd.DataFrame.from_dict(d)
    snapdeal_df['title'].replace('', np.nan, inplace=True)
    snapdeal_df = snapdeal_df.dropna(subset=['title'])
    snapdeal_df.to_csv("snapdeal_data.csv", header=True, index=False)

In [12]:
snapdeal_df

,title,price,rating,description,sellername
0,RESIST EYEWEAR - White Rectangular Sunglasses ...,849,(5.0),POLY CARBONATE LENS : 100% UV Protection - 100...,vl enterprise llp
1,Fair-X - Gold Pilot Sunglasses ( Pack of 1 ),209,(4.2),The images represent actual product though col...,arizona fo
2,Creature - Black Square Sunglasses ( Pack of 2 ),379,(4.1),Creature Black & Green Sunglasses Combo with U...,creature retail pvt ltd
3,Style Smith Unisex Pilot Polarized & UV Protec...,399,(3.8),Style Smith Sunglasses collection features som...,juscorp enterprises private limited
4,David Martin Black Square Sunglasses ( Pack of...,129,(3.9),"Retro Square Sunglasses (For Men & Women, Black)",rajeev optics
5,Kanny Devis Black Oval Sunglasses ( Pack of 1 ),259,(4.5),Animal Movie Inspired Sunglasses for Men : Ste...,az-aarjav
6,Peter Jones - Black Square Sunglasses ( Pack o...,749,(4.4),The images represent actual product though col...,chawla traders
7,Funk Gold Pilot Sunglasses ( Pack of 1 ),129,,The images represent actual product though col...,rajeev optics
8,Arizona Sunglasses - Black Rectangular Sunglas...,339,(4.2),The images represent actual product though col...,az-aarjav
9,RESIST EYEWEAR - Black Pilot Sunglasses ( Pack...,1199,(4.5),"Top of Form, currently one of the most iconic ...",vl enterprise llp
